In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
from sqlalchemy import create_engine, func

In [2]:
games = pd.read_csv("C:/Users/jadev/UCD/Homework/ETL_Project/video_games.txt")
meta = pd.read_csv("C:/Users/jadev/UCD/Homework/ETL_Project/archive.zip")

In [3]:
games.head()

,number,game,release_date,price,owners,developer,publisher,average_playtime,median_playtime,metascore
0,1,Half-Life 2,"Nov 16, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,110.0,66.0,96.0
1,3,Counter-Strike: Source,"Nov 1, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,236.0,128.0,88.0
2,21,Counter-Strike: Condition Zero,"Mar 1, 2004",9.99,"10,000,000 .. 20,000,000",Valve,Valve,10.0,3.0,65.0
3,47,Half-Life 2: Deathmatch,"Nov 1, 2004",4.99,"5,000,000 .. 10,000,000",Valve,Valve,0.0,0.0,NaN
4,36,Half-Life: Source,"Jun 1, 2004",9.99,"2,000,000 .. 5,000,000",Valve,Valve,0.0,0.0,NaN


In [4]:
meta.head()

,metascore,name,console,userscore,date
0,97,Grand Theft Auto V,PS3,8.3,"Sep 17, 2013"
1,97,Grand Theft Auto V,X360,8.3,"Sep 17, 2013"
2,95,The Last of Us,PS3,9.2,"Jun 14, 2013"
3,94,BioShock Infinite,PS3,8.5,"Mar 26, 2013"
4,94,BioShock Infinite,PC,8.6,"Mar 25, 2013"


In [5]:
games.dtypes

number                int64
game                 object
release_date         object
price               float64
owners               object
developer            object
publisher            object
average_playtime    float64
median_playtime     float64
metascore           float64
dtype: object

In [6]:
meta.dtypes

metascore     int64
name         object
console      object
userscore    object
date         object
dtype: object

In [7]:
meta['userscore'] = pd.to_numeric(meta['userscore'],errors='coerce')
meta['date'] = pd.to_datetime(meta['date'])

In [8]:
meta.head()

,metascore,name,console,userscore,date
0,97,Grand Theft Auto V,PS3,8.3,2013-09-17
1,97,Grand Theft Auto V,X360,8.3,2013-09-17
2,95,The Last of Us,PS3,9.2,2013-06-14
3,94,BioShock Infinite,PS3,8.5,2013-03-26
4,94,BioShock Infinite,PC,8.6,2013-03-25


In [9]:
games['release_date'] = pd.to_datetime(games['release_date'])

In [10]:
games.head()

,number,game,release_date,price,owners,developer,publisher,average_playtime,median_playtime,metascore
0,1,Half-Life 2,2004-11-16,9.99,"10,000,000 .. 20,000,000",Valve,Valve,110.0,66.0,96.0
1,3,Counter-Strike: Source,2004-11-01,9.99,"10,000,000 .. 20,000,000",Valve,Valve,236.0,128.0,88.0
2,21,Counter-Strike: Condition Zero,2004-03-01,9.99,"10,000,000 .. 20,000,000",Valve,Valve,10.0,3.0,65.0
3,47,Half-Life 2: Deathmatch,2004-11-01,4.99,"5,000,000 .. 10,000,000",Valve,Valve,0.0,0.0,NaN
4,36,Half-Life: Source,2004-06-01,9.99,"2,000,000 .. 5,000,000",Valve,Valve,0.0,0.0,NaN


In [11]:
meta_game = meta.rename(columns={'name':'game'})

In [12]:
meta_game.dtypes

metascore             int64
game                 object
console              object
userscore           float64
date         datetime64[ns]
dtype: object

In [13]:
games.dtypes

number                       int64
game                        object
release_date        datetime64[ns]
price                      float64
owners                      object
developer                   object
publisher                   object
average_playtime           float64
median_playtime            float64
metascore                  float64
dtype: object

In [14]:
games_clean = games[games['metascore'].notna()]
games_clean_dup = games_clean.drop_duplicates(subset=['game'])
games_no_null = games_clean_dup.dropna(subset=['game'])

In [15]:
#I'm looking to buy a PSVita and would like to find the best ones to purchase
meta_ps = meta_game[(meta_game.console=='VITA')]
meta_ps

,metascore,game,console,userscore,date
20,88,Spelunky,VITA,7.9,2013-08-27
33,87,Guacamelee!,VITA,8.3,2013-04-09
36,87,Rayman Legends,VITA,8.4,2013-09-03
37,87,Tearaway,VITA,8.9,2013-11-22
46,87,Velocity Ultra,VITA,7.9,2013-07-02
...,...,...,...,...,...
14302,63,Operation Babel: New Tokyo Legacy,VITA,4.0,2017-05-16
14477,55,A.W.: Phoenix Festa,VITA,4.5,2016-07-26
15308,58,The Caligula Effect,VITA,7.0,2017-05-02
15637,39,Drive Girls,VITA,5.5,2017-09-08


In [16]:
connection_string = 'postgres:class_time@localhost:5432/etl-challenge'
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
engine.table_names()

['games', 'meta_game']

In [18]:
games_no_null.to_sql(name='games', con = engine, if_exists = 'append',index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "games_pkey"
DETAIL:  Key (game)=(Half-Life 2) already exists.

[SQL: INSERT INTO games (number, game, release_date, price, owners, developer, publisher, average_playtime, median_playtime, metascore) VALUES (%(number)s, %(game)s, %(release_date)s, %(price)s, %(owners)s, %(developer)s, %(publisher)s, %(average_playtime)s, %(median_playtime)s, %(metascore)s)]
[parameters: ({'number': 1, 'game': 'Half-Life 2', 'release_date': datetime.datetime(2004, 11, 16, 0, 0), 'price': 9.99, 'owners': '10,000,000\xa0..\xa020,000,000', 'developer': 'Valve', 'publisher': 'Valve', 'average_playtime': 110.0, 'median_playtime': 66.0, 'metascore': 96.0}, {'number': 3, 'game': 'Counter-Strike: Source', 'release_date': datetime.datetime(2004, 11, 1, 0, 0), 'price': 9.99, 'owners': '10,000,000\xa0..\xa020,000,000', 'developer': 'Valve', 'publisher': 'Valve', 'average_playtime': 236.0, 'median_playtime': 128.0, 'met

In [ ]:
meta_ps.to_sql(name='meta_game', con = engine, if_exists = 'append',index=False)